# Scope and lifetime of objects

The contiguous region of the program where an identifier (name) can be accessed is called the *scope* of the identifier. C and Java share similarities in that blocks determine the scope of identifiers, but there are
different kinds of scope in the two languages.

The lifetime of an object determines when it is valid to use the object. In Java, the programmer usually is
not concerned about object lifetime because the language uses garbage collection to release memory used by
objects that can no longer be accessed. Furthermore, there are no pointers in Java which means that it is
usually impossible to obtain a reference to a "dead" object. In C, the existance of pointers and the ability
to dynamically allocate and deallocate memory means that the programmer must always be aware of object lifetime
issues.

This notebook mentions the term *linkage*. See the *Linkage* notebook for details.

## Scope

There are four kinds of scope in C:

* file scope
* block scope
* function prototype scope
* function scope

Function scope is relevant only to labels declared inside of a function and is not discussed in this notebook.

#### File scope

If an identifier is declared outside of any block or parameter list then it has *file scope*. The identifier
is usable at the point of declaration and ends at the end of the translation unit. Furthermore, other
translation units may be able to link to such an identifier if the identifier has external linkage.

The following program has three identifiers with file scope:

1. the variable `j`
2. the function `f`
3. the function `main`

In [1]:
// scope.c

#include <stdio.h>

int j;                                                       // j has file scope

void f(int i) {                                              // f has file scope
    int j = 1;
    i++;
    printf("\tfunc: i = %d, j = %d\n", i, j);
    for (int i = 0; i < 2; i++) {
        int j = 2;
        printf("\t\tfor loop, i = %d, j = %d\n", i, j);
    }
    printf("\tfunc: i = %d, j = %d\n", i, j);
}

int main(void) {                                             // main has file scope
    printf("main: j = %d\n", j);
    f(100);
    printf("main: j = %d\n", j);

    return 0;
}


main: j = 0
	func: i = 101, j = 1
		for loop, i = 0, j = 2
		for loop, i = 1, j = 2
	func: i = 101, j = 1
main: j = 0


#### Block scope

Braces `{ }` denote blocks of code (similar to Java).
If an identifier is declared inside of a block or in a parameter list, then the identifier has *block scope*.
The identifier is accessible everywhere in the block it is declared in after the point where it is declared.

Blocks appear inside of files and inside of other blocks which causes their scopes to be nested.
It is legal, but often confusing, to declare identifiers with the same name in different scopes.
An identifier declared at an inner scope takes precedence over an identifier declared at an outer
scope. We say that the other scope identifier is *hidden* at the inner scope.

#### Function prototype scope

If a function is declared, but not defined, then the parameters of the declared function have
*function prototype scope*. 

In [2]:
// function prototype
// parameter a has function prototype scope
void some_function(int a);  

Usually, function prototype scope is not very interesting but it can affect the order in
which the parameters must appear. The following prototype is correct:

In [3]:
// function prototype
// parameters n and a have function prototype scope
void another_function(int n, int a[n]);  

The following prototype is incorrect because the parameter `n` is used before it comes into scope:

In [4]:
// function prototype
// parameters n and a have function prototype scope
// error because n is used before it is in scope
void an_incorrect_function(int a[n], int n);  

/tmp/tmpk_iip2ju.c: In function ‘main’:
/tmp/tmpk_iip2ju.c:6:34: error: ‘n’ undeclared (first use in this function)
    6 | void an_incorrect_function(int a[n], int n);
      |                                  ^
/tmp/tmpk_iip2ju.c:6:34: note: each undeclared identifier is reported only once for each function it appears in
[C kernel] GCC exited with code 1, the executable will not be executed

## Lifetime

The lifetime of an object is determined by its *storage duration*. There are four kinds of storage duration in C:

1. automatic
2. static
3. allocated
4. thread

Thread storage duration is not covered in CISC220 but may be relevant in CISC324 Operating Systems.

#### Automatic storage duration

Automatic storage duration is the default storage duration of an object declared within a block or as a function
parameter. Memory for the object is allocated when the block in which the object was declared is entered
and deallocated when the block is exited by any means.